In [4]:
import requests
from bs4 import BeautifulSoup
import json
import time
import urllib
import pandas as pd

In [5]:
games = pd.read_csv('steam_coops.csv', index_col=0)

In [6]:
games.head(10)

,game_id,title,release_date,price,rating_inPercent,url
0,1426210,It Takes Two,"25 Mar, 2021",2499.0,95.0,https://store.steampowered.com/app/1426210/It_...
1,1172470,Apex Legends™,"4 Nov, 2020",0.0,90.0,https://store.steampowered.com/app/1172470/Ape...
2,1085660,Destiny 2,"1 Oct, 2019",0.0,85.0,https://store.steampowered.com/app/1085660/Des...
3,1056640,Phantasy Star Online 2 New Genesis,"5 Aug, 2020",0.0,80.0,https://store.steampowered.com/app/1056640/Pha...
4,252490,Rust,"8 Feb, 2018",725.0,86.0,https://store.steampowered.com/app/252490/Rust...
5,381210,Dead by Daylight,"14 Jun, 2016",499.0,82.0,https://store.steampowered.com/app/381210/Dead...
6,892970,Valheim,"2 Feb, 2021",435.0,96.0,https://store.steampowered.com/app/892970/Valh...
7,739630,Phasmophobia,"18 Sep, 2020",309.0,97.0,https://store.steampowered.com/app/739630/Phas...
8,552990,World of Warships,"15 Nov, 2017",0.0,81.0,https://store.steampowered.com/app/552990/Worl...
9,1097150,Fall Guys: Ultimate Knockout,"3 Aug, 2020",465.0,83.0,https://store.steampowered.com/app/1097150/Fal...


In [7]:
def get_url(old_link):
    #collects URL with Steam app id
    
    parts = old_link.split('/')[3:5]
    return "https://steamcommunity.com/"+ parts[0] + "/" + parts[1] + "/" +"reviews/?browsefilter=toprated&snr=1_5_100010_#scrollTop=100"

JSON structure from: https://partner.steamgames.com/doc/store/getreviews

In [8]:
def get_reviews(appid):
    cursor = '*'
    url = f"https://store.steampowered.com/appreviews/{appid}?json=1&filter=updated&language=english&cursor={cursor}&num_per_page=100"
    response = requests.get(url)
    y = json.loads(response.content.decode('utf-8'))
    
    reviews_list = [] #init empty reviews list
    pos = lambda x: 1 if x == True else 0
    if y['query_summary']['total_reviews'] >= 5000: #game has more than 5000 revs
        #print("game has ", y['query_summary']['total_reviews'], " reviews")
        while len(reviews_list) < 5000: #if we collected less than 5000 reviews
            num_revs = y['query_summary']["num_reviews"]
            all_rews = y['reviews'] #take all revs
            cursor = urllib.parse.quote_plus(y['cursor'])#we need to URL encode cursor
            #print(num_revs, ' ', len(reviews_list))
            #print('cursor encoded ', cursor)
            for item in all_rews:
                g = {'game id':appid,
                    'review text': item['review'],
                    'positive' : pos(item['voted_up'])
                    }
                reviews_list.append(g)
            url = f"https://store.steampowered.com/appreviews/{appid}?json=1&filter=updated&language=english&cursor={cursor}&num_per_page=100"
            response = requests.get(url) 
        time.sleep(3)
    #else:
        #print("NaN")
    return reviews_list

In [ ]:
n = 0
reviews = []
for id in games['game_id'].tolist():
    print('---------GAME', id, ' ', n)
    reviews += get_reviews(id)
    n += 1

In [33]:
df = pd.DataFrame(reviews)
df

In [18]:
df.to_csv('reviews_demonstration.csv')

In [5]:
reviews = pd.read_csv('reviews_demonstration.csv', index_col=0)
reviews = reviews.dropna()
reviews.loc[0, 'review text']

"The game itself was easy to play, which is best even for those who doesn't usually play video games but want to have a good time with their friends/partner. Tho it was simple, the pacing, art, music, and variety of game play completely makes up for what it lakes in difficulty. I recommend this game to everyone."